In [1]:
# May need to install any of the following:
# pip install requests
# pip install bs4
# pip install pandas
# pip install sqlalchemy

# Import the libraries we will use to scrape symbol data
import io
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sqlalchemy as sa
import sqlite3
from sqlite3 import Error
from datetime import date, datetime, timezone, timedelta
from dateutil.parser import parse

import os
import glob
import re
import csv

import logging
import random

In [2]:
#=========================================================================================

# Define required functions

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [3]:
#=========================================================================================

# Ensure our output directory exists
if not os.path.exists('output'):
    os.makedirs('output')

In [4]:
conn = create_connection("output/team122project.sqlite3")

df_current = pd.read_sql_query("SELECT * FROM securities_by_date ORDER BY DATE", conn, index_col='Date')
df_current.head()

df_no_all_nulls = df_current.drop(df_current[df_current.any(axis=1) == False].index, axis=0)
df_no_all_nulls.head(20)

print(f"Current Size: {df_current.shape[0]}, New size: {df_no_all_nulls.shape[0]}")

df_no_all_nulls.to_sql(name='securities_by_date', con=conn, if_exists='replace', index=True)

In [5]:
#=========================================================================================

cur = conn.cursor()
cur.execute("SELECT EXISTS(SELECT 1 FROM sqlite_master WHERE type='table' AND name='securities_by_date') AS securities_exist")
securities_exist, = cur.fetchone()

if bool(securities_exist):
    cur.execute("SELECT Date AS LastUpdate FROM securities_by_date ORDER BY Date DESC LIMIT 1")
    last_update, = cur.fetchone()
else:
    last_update = '2009-12-31T00:00:00' # Hard code to the day before our earliest date to be considered for this project

print(last_update)

2023-10-18T00:00:00


In [6]:
dt_last_update = datetime.fromisoformat(last_update)
dt_last_update.year, dt_last_update.month, dt_last_update.day

(2023, 10, 18)

In [7]:
#dt_now = date.today() - timedelta(days=1)
# For update purposes, do not subtract one day. In fact, there must always be
# at least one day difference between the start and edn dates according to API
dt_now = date.today()
dt_now = datetime(dt_now.year, dt_now.month, dt_now.day, tzinfo=timezone.utc)
dt_last_update = dt_last_update + timedelta(days=1)
dt_last_update = datetime(dt_last_update.year, dt_last_update.month, dt_last_update.day, tzinfo=timezone.utc)


In [8]:
timestmp1 = int(dt_last_update.timestamp())
timestmp2 = int(dt_now.timestamp())

print(f"\nYahoo period1 = {timestmp1} corresponds to date {dt_last_update}")
print(f"\nYahoo period2 = {timestmp2} corresponds to date {dt_now}")


Yahoo period1 = 1697673600 corresponds to date 2023-10-19 00:00:00+00:00

Yahoo period2 = 1697760000 corresponds to date 2023-10-20 00:00:00+00:00


In [9]:
#=========================================================================================

datatypes = {
      'Country'             : str
    , 'Exchange_ID'         : str
    , 'Symbol'              : str
    , 'Description'         : str
    , 'Local_Symbol'        : str
    , 'IPO_Date'            : str
    , 'Category1'           : str
    , 'Category2'           : str
    , 'Category3'           : str
    , 'GICS_Sector'         : str
    , 'ISIN'                : str
    , 'SEDOL'               : str
    , 'Market_Cap'          : str #np.int64
    , 'Currency'            : str
    , 'Actions'             : str
    , 'Yahoo_Symbol'        : str
    , 'Yahoo_Listings_Link' : str
}

datecolumns = ['IPO_Date']

# Read the links from our SQLite3 database

df_Yahoo_links_good_sectors_latestdates = pd.read_sql('select * from yahoo_links', con=conn, dtype=datatypes, parse_dates=datecolumns) #, date_format='mm/dd/yyyy')

#logging.info(f"\nRead {df_Yahoo_links_good_sectors_latestdates.shape[0]} rows.\n{df_Yahoo_links_good_sectors_latestdates.head()}")


In [10]:
#=========================================================================================

# Create a series of dates from the latest date to be updated to yesterday
idxDates = pd.Series(pd.date_range(start=dt_last_update.date(), end=date(dt_now.year, dt_now.month, dt_now.day)))
# Create an empty dataframe with just the index (= the series of dates)
dfAllDates = pd.DataFrame(index=idxDates)
# Rename the index
dfAllDates.index.name="Date"

print(f"\nFirst 10 dates in dataframe dfAllDates:\n{dfAllDates.head(10)}")

print(f"\nLast 10 dates in dataframe dfAllDates:\n{dfAllDates.tail(10)}")



First 10 dates in dataframe dfAllDates:
Empty DataFrame
Columns: []
Index: [2023-10-19 00:00:00, 2023-10-20 00:00:00]

Last 10 dates in dataframe dfAllDates:
Empty DataFrame
Columns: []
Index: [2023-10-19 00:00:00, 2023-10-20 00:00:00]


In [11]:
# -----------------------------------------------------------------------------------------------------------------------
# This is where the Yahoo downloads actually occur - may need to use random proxy URLs to get past Yahoo's request limits
# -----------------------------------------------------------------------------------------------------------------------

loopUntil = df_Yahoo_links_good_sectors_latestdates.shape[0]
####################################################################
####################################################################
####################################################################
loopUntil = 4 # Try with a smaller set of data, instead of all files
####################################################################
####################################################################
####################################################################
dict_dfYahooSecurities = {}

with requests.session():
    header = {
        'Connection': 'keep-alive',
        'Expires': '-1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
    }

    for i in range(loopUntil):
        symbol, link = df_Yahoo_links_good_sectors_latestdates.iloc[i][["Yahoo_Symbol", "Yahoo_Listings_Link"]]

        # Replace the placeholders in our link with actual timestamps
        link = link.replace('{timestmp1}', str(timestmp1)).replace('{timestmp2}', str(timestmp2))

        #proxy_url = random.choice(lst_good_proxies_url)
        #logging.info(f"\nAttempting download historical prices for symbol '{symbol}'' from {link} using proxy {proxy_url}")
        print(f"\nAttempting download historical prices for symbol '{symbol}'' from {link}")
        try:
            #dict_dfYahooSecurities[symbol] = pd.read_csv(link, usecols=["Date", "Adj Close"], parse_dates=True, infer_datetime_format=True)
            # We cannot use read_csv directly, because Yahoo will block our requests if we issue too many from the same IP address, so...
            #response = requests.get(link, proxies={"http": proxy_url, "https": proxy_url})
            response = requests.get(link, headers=header)
            #logging.info(f"\n{response.text[:200]}")
            #logging.info(f"\n{response.text[-200:]}")
            dict_dfYahooSecurities[symbol] = pd.read_csv(io.StringIO(response.text), usecols=["Date", "Adj Close"])
            # Convert the Date column to the correct data type
            dict_dfYahooSecurities[symbol]['Date'] = pd.to_datetime(dict_dfYahooSecurities[symbol]['Date'], utc=False) #, utc=True
            dict_dfYahooSecurities[symbol].columns = ["Date", symbol]
            dict_dfYahooSecurities[symbol].set_index('Date', drop=True, inplace=True)

            # We don't need to save to CSV since we write directly to table in SQLite3 database now
            """
            outputfilename =f"output/history_security_{symbol}.csv"
            with open(outputfilename, "w") as filehandle:
                #dict_dfYahooSecurities[symbol].to_csv(filehandle, index=False, lineterminator = '\r', encoding='utf-8-sig')
                dict_dfYahooSecurities[symbol].to_csv(filehandle, index=False, lineterminator = '\n', encoding='utf-8')
            if not filehandle.closed:
                filehandle.close()
            filehandle = None
            """
        except Exception as e:
            #logging.exception(str(e))
            print(str(e))
            continue

print("\nJoining all columns of the downloaded securities into a single dataframe")

for df in dict_dfYahooSecurities.values():
    dfAllDates = dfAllDates.join(df)


Attempting download historical prices for symbol 'AAMC'' from https://query1.finance.yahoo.com/v7/finance/download/AAMC?period1=1697673600&period2=1697760000&interval=1d&events=history&includeAdjustedClose=true

Attempting download historical prices for symbol 'ACU'' from https://query1.finance.yahoo.com/v7/finance/download/ACU?period1=1697673600&period2=1697760000&interval=1d&events=history&includeAdjustedClose=true

Attempting download historical prices for symbol 'AE'' from https://query1.finance.yahoo.com/v7/finance/download/AE?period1=1697673600&period2=1697760000&interval=1d&events=history&includeAdjustedClose=true

Attempting download historical prices for symbol 'AGE'' from https://query1.finance.yahoo.com/v7/finance/download/AGE?period1=1697673600&period2=1697760000&interval=1d&events=history&includeAdjustedClose=true

Joining all columns of the downloaded securities into a single dataframe


In [12]:
dfAllDates

,AAMC,ACU,AE,AGE
Date,,,,
2023-10-19,6.55,28.049999,31.26,0.652
2023-10-20,NaN,NaN,NaN,NaN


In [13]:
dfToAppend = dfAllDates[dfAllDates.any(axis=1)]
dfToAppend

,AAMC,ACU,AE,AGE
Date,,,,
2023-10-19,6.55,28.049999,31.26,0.652


In [14]:
print(f"\nSaving joined dataframe to table 'securities_by_date' in database 'output/team122project.sqlite3'")
dfToAppend.to_sql(name='securities_by_date', con=conn, if_exists='append', index=True)


Saving joined dataframe to table 'securities_by_date' in database 'output/team122project.sqlite3'


1